In [ ]:
from copy import deepcopy
import json
import time
import globus_automate_client
import mdf_toolbox
import os
os.environ['FLASK_ENV']='development'
from aws.globus_automate_flow import GlobusAutomateFlow, FlowAction
from globus_sdk import ClientCredentialsAuthorizer, AccessTokenAuthorizer

import globus_sdk
from globus_automate_client import FlowsClient



In [ ]:
MANAGE_FLOWS_SCOPE = "https://auth.globus.org/scopes/eec9b274-0c81-4334-bdc2-54e90e689b9a/manage_flows"


def authorizer_callback(*args, **kwargs):
    auth = AccessTokenAuthorizer(
        tokens.by_resource_server[mdf_flow.flow_id]['access_token']
    )
    return auth

with open(".mdfsecrets", 'r') as f:
    globus_secrets = json.load(f)
    
mdf_flow = GlobusAutomateFlow.from_existing_flow("mdf_flow_info.json")
conf_client = globus_sdk.ConfidentialAppAuthClient(
            globus_secrets['API_CLIENT_ID'],
            globus_secrets['API_CLIENT_SECRET'])

requested_scopes = [
    "https://auth.globus.org/scopes/eec9b274-0c81-4334-bdc2-54e90e689b9a/manage_flows",
    "https://auth.globus.org/scopes/eec9b274-0c81-4334-bdc2-54e90e689b9a/view_flows",
    "https://auth.globus.org/scopes/eec9b274-0c81-4334-bdc2-54e90e689b9a/run",
    "https://auth.globus.org/scopes/eec9b274-0c81-4334-bdc2-54e90e689b9a/run_status",
    mdf_flow.flow_scope
]

tokens = conf_client.oauth2_client_credentials_tokens(
    requested_scopes=requested_scopes)

cca = ClientCredentialsAuthorizer(
    conf_client,
    MANAGE_FLOWS_SCOPE,
    tokens.by_resource_server['flows_automated_tests']['access_token'],
    tokens.by_resource_server['flows_automated_tests']['expires_at_seconds']
)

flows_client = FlowsClient.new_client(
    client_id=globus_secrets['API_CLIENT_ID'],
    authorizer_callback=authorizer_callback,
    authorizer=cca)


In [ ]:
flow = GlobusAutomateFlow.from_existing_flow("mdf_flow_info.json")
action = FlowAction(flow, "096382b6-a820-4b6a-a1bc-5b188db3d82b")
flow.set_client(flows_client)

In [ ]:
action.get_status()

In [ ]:
# Get test input for Flow
with open("mdf_flow_input.json") as f:
    flow_input = json.load(f)

# Add credentials to input
flow_input["_private_feedstock_auth_header"] = feedstock_auth_header
flow_input["_tokens"] = {"MDFUser": run_as_token}

# Create unique source_id for submission (source_id must be unique except for updates)
source_id = "mdf_flow_test_" + str(int(time.time())) + "_v1.1"
flow_input["source_id"] = source_id

In [ ]:
# Initiate Flow
# Careful to run this cell only once per test - the first Flow's Action ID will be overwritten and lost
# if this is run a second time
flow_res = flows_client.run_flow(flow_id, flow_scope, flow_input)
flow_res.data

In [ ]:
# If looking at a flow started elsewhere
# Alternatively, to keep the same Flow ID and scope, update the Flow
# Deploying recommended over updating - Automate Flow updates are not stable
with open("mdf_flow_info.json") as f:
    flow_info = json.load(f)
    flow_id = flow_info["flow_id"]
    flow_scope = flow_info["flow_scope"]

action_id = "69a7cff3-9ca8-41e0-a998-e4378051f5df"
print(flow_info)

In [ ]:
# Check status of Flow
# This cell should be run multiple times, to check the status until the Flow succeeds or fails
status = flows_client.flow_action_status(flow_id, flow_scope, action_id).data
print(json.dumps(status, indent=4, sort_keys=True))

In [ ]:
# Check the Flow Log
# The Log is very verbose and only necessary when debugging errors in Flow execution
flows_client.flow_action_log(flow_id, flow_scope, flow_res["action_id"], limit=100).data